In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("model train") \
    .config("spark.executor.memory", "8G") \
    .getOrCreate()


sc = spark.sparkContext

24/05/05 10:19:08 WARN Utils: Your hostname, QINGHAOdeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.136.26.3 instead (on interface en0)
24/05/05 10:19:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/05 10:19:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import sys
sys.path.append("..")
import data_processing.data_process as data_process

train_data_path = '../data/traindata.csv'
test_data_path = '../data/testdata.csv'
train_data_processed, test_data_processed = data_process.operate(spark, train_data_path, test_data_path)

train_data_processed.show(5)

24/05/05 10:19:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------+------------------+-----------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+-------------------------+-------------------------+-------------------------+-------------------------+--------------

In [3]:
# Cache the DataFrame
train_data_processed.cache()

DataFrame[separation real: double, propensity real: double, length real: double, PredSS_freq_central_H real: double, PredSS_freq_central_E real: double, PredSS_freq_central_C real: double, PredCN_freq_central_0 real: double, PredCN_freq_central_1 real: double, PredCN_freq_central_2 real: double, PredCN_freq_central_3 real: double, PredCN_freq_central_4 real: double, PredRCH_freq_central_0 real: double, PredRCH_freq_central_1 real: double, PredRCH_freq_central_2 real: double, PredRCH_freq_central_3 real: double, PredRCH_freq_central_4 real: double, PredSA_freq_central_0 real: double, PredSA_freq_central_1 real: double, PredSA_freq_central_2 real: double, PredSA_freq_central_3 real: double, PredSA_freq_central_4 real: double, PredSS_freq_global_H real: double, PredSS_freq_global_E real: double, PredSS_freq_global_C real: double, PredCN_freq_global_0 real: double, PredCN_freq_global_1 real: double, PredCN_freq_global_2 real: double, PredCN_freq_global_3 real: double, PredCN_freq_global_4 

In [4]:
# Feature selection

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.feature import StandardScaler

label_col = "class {0,1}"
feature_cols = train_data_processed.columns
feature_cols.remove('class {0,1}')



In [5]:
# Build vectorAssembler
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Feature Selection with ChiSqSelector
selector = ChiSqSelector(percentile=0.1, featuresCol="features", outputCol="select_features", labelCol=label_col)


In [6]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline

# Build a GBT model
gbt = GBTClassifier(featuresCol="select_features", labelCol=label_col, seed=42)

# Creat Pipeline
pipeline = Pipeline(stages=[vector_assembler, selector, gbt])

# Create parameter grid
param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [10, 20]) \
    .addGrid(gbt.maxDepth, [2, 4, 6]) \
    .addGrid(gbt.stepSize, [0.1, 0.01]) \
    .build()

# evaluator = BinaryClassificationEvaluator(labelCol=label_col, metricName="areaUnderROC")
evaluator = MulticlassClassificationEvaluator(labelCol=label_col, metricName="accuracy")

# Create cross-validator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=3)

# Fit cross-validation model
cv_model = crossval.fit(train_data_processed)

# Get best model from cross-validation
best_model = cv_model.bestModel

# Get best parameters
best_params = best_model.stages[-1].extractParamMap()

print("Best Parameters:")
for param, value in best_params.items():
    print(f"{param.name}: {value}")

# Use evaluators to evaluate models
acc = evaluator.evaluate(cv_model.transform(test_data_processed))
print("Accuracy:", acc)

#auc = evaluator.evaluate(cv_model.transform(test_df))
#print("Area Under ROC:", auc)

24/05/05 10:19:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/05 10:19:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Best Parameters:
cacheNodeIds: False
checkpointInterval: 10
featureSubsetStrategy: all
featuresCol: select_features
impurity: variance
labelCol: class {0,1}
leafCol: 
lossType: logistic
maxBins: 32
maxDepth: 4
maxIter: 20
maxMemoryInMB: 256
minInfoGain: 0.0
minInstancesPerNode: 1
minWeightFractionPerNode: 0.0
predictionCol: prediction
probabilityCol: probability
rawPredictionCol: rawPrediction
seed: 42
stepSize: 0.1
subsamplingRate: 1.0
validationTol: 0.01
Accuracy: 0.952


In [7]:
gbt = GBTClassifier(maxIter=20, maxDepth=4, stepSize=0.01, featuresCol="select_features", labelCol=label_col)

pipeline = Pipeline(stages=[vector_assembler, selector, gbt])
model = pipeline.fit(train_data_processed)

In [19]:
train_df = train_data_processed.toPandas()
test_df = test_data_processed.toPandas()
# print(train_df)
X_train = train_df.loc[:, test_df.columns != 'class {0,1}']
y_train = train_df['class {0,1}']
X_test = test_df.loc[:, test_df.columns != 'class {0,1}']
y_test = test_df['class {0,1}']


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
gbt_classifier = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=4, random_state=42)
gbt_classifier.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=4, n_estimators=20, random_state=42)